In [1]:
%cd hypernet

/home/z1157095/hypernet-cnn/hypernet


In [2]:
from dotenv import load_dotenv
load_dotenv()

import os

In [3]:
from comet_ml import Experiment, Optimizer

import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
import torch.utils.data as data_utils
import pandas as pd
from collections import defaultdict

torch.set_default_dtype(torch.float32)

In [4]:
from torchsummary import summary
import matplotlib.pyplot as plt
from tqdm import tqdm, trange

In [8]:
import tabular_hypernet as hp

In [11]:
hp.training_utils.get_dataset

<function tabular_hypernet.training_utils.get_dataset(size=60000, masked=False, mask_no=200, mask_size=700, shared_mask=False, batch_size=32, test_batch_size=32)>

In [ ]:
os.environ['COMET_KEY'] = 'UXrV5UxyhTK3cyQNG6BDuc4bE'
os.environ.get("COMET_KEY")

In [18]:
hp.semisl.get_dataset

<function tabular_hypernet.semisl.get_dataset(size=(100, 900), masked=False, mask_no=200, mask_size=700, shared_mask=False, batch_size=32, test_batch_size=32)>

In [63]:
class TabSSLCrossEntropyLoss(torch.nn.Module):
    def __init__(self, beta=0.1, unsup_target_wrapper=torch.nn.functional.softmax, threshold=None):
        super(TabSSLCrossEntropyLoss, self).__init__()
        
        self.y_f1 = torch.nn.CrossEntropyLoss()
        self.y_f2 = torch.nn.CrossEntropyLoss()
        
        self.f1_f2 = torch.nn.CrossEntropyLoss()
        self.f2_f1 = torch.nn.CrossEntropyLoss()
        
        self.beta = beta
        self.unsup_target_wrapper = unsup_target_wrapper
        self.threshold = threshold
    
    def forward(self, sup_input, unsup_input):
        sup_outputs1, sup_outputs2, sup_labels = sup_input
        unsup_outputs1, unsup_outputs2 = unsup_input
        
        self.supervised_loss = self.y_f1(sup_outputs1, sup_labels) + self.y_f2(sup_outputs2, sup_labels)
        
        self.self_supervised_loss = 0
        if self.threshold:
            unsup_outputs1_target = torch.nn.functional.softmax(unsup_outputs1, dim=1)
            mask1 = unsup_outputs1_target >= self.threshold
            
            if len(unsup_outputs1_target[mask1]):
                print(unsup_outputs1_target[mask1], flush=True)
                unsup_outputs1_target = torch.argmax(unsup_outputs1_target[mask1], dim=1)
                self.self_supervised_loss += self.f1_f2(unsup_outputs1[mask2], unsup_outputs2_target)
            
            unsup_outputs2_target = torch.nn.functional.softmax(unsup_outputs2, dim=1)
            mask2 = unsup_outputs2_target >= self.threshold
            
            if len(unsup_outputs2_target[mask2]):
                print(unsup_outputs2_target, flush=True)
                print(unsup_outputs2_target[mask2], flush=True)
                unsup_outputs2_target = torch.argmax(unsup_outputs2_target[mask2], dim=1)
                self.self_supervised_loss += self.f2_f1(unsup_outputs2[mask1], unsup_outputs1_target)
        
        else:
            self.self_supervised_loss = self.f1_f2(unsup_outputs1, self.unsup_target_wrapper(unsup_outputs2, dim=1)) \
                                + self.f2_f1(unsup_outputs2, self.unsup_target_wrapper(unsup_outputs1, dim=1))
        
        return self.supervised_loss + self.beta * self.self_supervised_loss


In [65]:
X = torch.tensor([[0.1, 0.5, -1.0], [0, 1.2, 0]])

In [66]:
X.requires_grad  = True

In [67]:
X = X * 1

In [68]:
X

tensor([[ 0.1000,  0.5000, -1.0000],
        [ 0.0000,  1.2000,  0.0000]], grad_fn=<MulBackward0>)

In [69]:
mask = X >= 0

In [70]:
mask

tensor([[ True,  True, False],
        [ True,  True,  True]])

In [71]:
indices = torch.nonzero(mask)
indices

tensor([[0, 0],
        [0, 1],
        [1, 0],
        [1, 1],
        [1, 2]])

In [72]:
X[indices]

IndexError: index 2 is out of bounds for dimension 0 with size 2

In [73]:
X[mask]

tensor([0.1000, 0.5000, 0.0000, 1.2000, 0.0000], grad_fn=<IndexBackward0>)

In [59]:
epochs = 100


mask_size = 100
masks_no = 50

results = defaultdict(list)
size = (100, 59900)

In [64]:
for beta in [0.]:
    for f in [torch.argmax]:
        criterion = hp.semisl.TabSSLCrossEntropyLoss(beta=beta, unsup_target_wrapper=f)

        hypernet = hp.Hypernetwork(mask_size=mask_size, node_hidden_size=100, test_nodes=masks_no).cuda()

        hypernet = hypernet.train()
        optimizer = torch.optim.Adam(hypernet.parameters(), lr=3e-4)

        # dataset & loaders
        sup_trainloader, unsup_trainloader, testloader = hp.semisl.get_dataset(size, batch_size=32, test_batch_size=64)
        trainloader = hp.semisl.TrainDataLoaderSemi(sup_trainloader, unsup_trainloader)

        results[size].append(hp.semisl.train_semisl(hypernet,
                                          optimizer,
                                          criterion,
                                          (trainloader, testloader), 
                                          size,
                                          epochs,
                                          masks_no,
                                          changing_beta=None,
                                          log_to_comet=False,
                                          tag='semisl_whole_dataset'))

  0%|                                                                                                                 | 0/100 [00:00<?, ?it/s]

tensor([[5.9393e-02, 2.0805e-02, 4.5298e-02, 1.0215e-01, 1.9716e-02, 1.8063e-02,
         3.9598e-03, 1.8617e-01, 2.6066e-01, 2.8379e-01],
        [3.8201e-01, 3.5600e-02, 2.4759e-01, 1.0786e-01, 8.6828e-03, 1.5643e-02,
         3.0427e-02, 3.1044e-02, 6.4663e-02, 7.6485e-02],
        [4.2504e-02, 5.4533e-01, 7.8627e-02, 4.5605e-02, 3.5453e-02, 3.5441e-02,
         4.5520e-02, 5.1149e-02, 2.8660e-02, 9.1711e-02],
        [5.0133e-02, 1.4162e-01, 6.4472e-02, 6.6837e-02, 2.2700e-01, 2.1753e-02,
         1.0158e-01, 1.2453e-01, 4.4120e-02, 1.5795e-01],
        [3.2829e-01, 1.3319e-03, 7.7331e-02, 7.4591e-02, 3.9633e-02, 3.0193e-03,
         1.4170e-02, 5.5594e-02, 3.0310e-01, 1.0294e-01],
        [3.8658e-02, 1.8514e-04, 9.1598e-01, 1.5943e-02, 9.7450e-04, 4.4046e-04,
         1.1793e-02, 2.3683e-04, 1.4591e-02, 1.2034e-03],
        [2.3874e-01, 4.8199e-02, 8.5324e-02, 2.7520e-01, 2.1499e-02, 5.9305e-03,
         8.3476e-03, 7.8078e-02, 7.2637e-02, 1.6605e-01],
        [6.4741e-02, 4.5717

  0%|                                                                                                                 | 0/100 [00:04<?, ?it/s]


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)